<a href="https://colab.research.google.com/github/jldz9/InSARScript/blob/tutorial/InSARScript_Tutorial_V110.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# InSARScript Tutorial
Welcome to InSARScript! InSAR Script is an open-source package designed to support the full InSAR processing pipeline. The primary goal of this package is to provide a streamlined and user-friendly InSAR processing experience across multiple satellite products.

This tutorial will guide you through the essential steps of using InSARScript

`⚠️ Note: This tutorial is for dev version of future InSARScript v1.1.0, it is not capable with old releases. `

## Setup Environment

This step will setup the program environment, this may take several minutes. Runtime restart might be required after installation.

Red warning is due to switching back to numpy 1.26.4 and can be ignored.

In [ ]:
!git clone https://github.com/jldz9/InSARScript.git
!pip install -e /content/InSARScript



## Account Setup
InSARScript requires multiple online accounts for full functionality. Registration for these accounts is completely free.

Register [NASA Earthdata](https://urs.earthdata.nasa.gov/)

Register [Copernicus Climate Data Store](https://cds.climate.copernicus.eu/)

This account is required for searching satellite scenes, downloading DEMs and orbit files, and submitting online interferogram processing jobs.
Once you've register your account please put your corresponding info below:

`  ⚠️ Note: This tutorial will not store or share your personal information`

In [ ]:
Earthdata_Username = 'Your_Earthdata_Username'
Earthdata_Password = 'Your_Earthdata_Password'

Once you registered Copernicus Climate Data Store account, Click [Here](https://cds.climate.copernicus.eu/how-to-api) to get your API token.

`  ⚠️ Note: Only insert your api key e.g.: b1f851fa-5a45-4521-8171-791d334f6ce3, do not include url`

In [ ]:
CCDS_API_Key = 'Your API Key'

In [ ]:
from pathlib import Path
import os
netrc_path = Path.home() / ".netrc"
# Create netrc file
rc_path = Path.home() / ".cdsapirc"
# Write credentials to ~/.netrc
with open(netrc_path, "w") as f:
    f.write(f"""machine urs.earthdata.nasa.gov
login {Earthdata_Username.strip()}
password {Earthdata_Password.strip()}
""")

with open(rc_path, "w") as f:
    f.write(f"""url: https://cds.climate.copernicus.eu/api
key: {CCDS_API_Key.strip()}
""")


---------------------------------------------------------Initial Setup Finished----------------------------------------------------------

## Workflow
The InSAR script is designed with three config-based main modules: **Downloader**, **Processor**, and **Analyzer** to cover the entire InSAR processing workflow:

**Set AOI** --> **Searching** --> **Result Filtering** --> **Interferogram** --> **Time-series Analysis**

### Set AOI

InSARScript allows to define the AOI using **bounding box**, **shapefiles**, or **WKT**:


- Bounding box

In [ ]:
AOI = [-113.05, 37.74, -112.68, 38.00]

- Shapefiles

In [ ]:
AOI = 'path/to/your/shapefile.shp'

- WKT

In [ ]:
AOI = 'POLYGON((-113.05 37.74, -113.05 38.00, -112.68 38.00, -112.68 37.74, -113.05 37.74))'

### Searching (Downloader)
Once the AOI is defined, you can perform searches using the Downloader.


In [ ]:
from insarscript import Downloader
AOI = [-113.05, 37.74, -112.68, 38.00]
s1 = Downloader.create('S1_SLC', intersectsWith=AOI)
results = s1.search()

Your AOI probably spans multiple scenes. To view the search result footprints, you can use:

In [ ]:
s1.footprint()

This will display a footprint map of the available Sentinel-1 scenes that covers the AOI. The stack indicates numbers of SAR sences in that footprint, becuase we have multiple stacks the graph will be a bit messy:

Let's check details of our SAR sence stacks and figure out which stack(s) we want to keep:

In [ ]:
s1.summary()

This will output the summary of availiable Sentinel-1 scenes that covers the AOI.

The program identified 18 potential stacks (14 ascending, 4 descending). We can narrowed the dataset to the descending track Path 100, Frame 466 in year 2020 by:

In [ ]:
filter_results = s1.filter(path_frame=(100,466), start='2020-01-01', end='2020-12-31')

Check back the footprint and summary:

In [ ]:
s1.footprint()
s1.summary()

## Interferogram (Processor)

After locating SAR scene stack(s), the next step is to generate unwrapped interferograms in preparation for the time-series analysis. InSARScript currently support:

- **HyP3**: Use the [HyP3 platform](https://hyp3-docs.asf.alaska.edu/)
 provided by ASF to run the interferometric processing in the cloud and download the resulting interferograms.

InSARSciprt has wrapped [hyp3_sdk](https://github.com/ASFHyP3/hyp3-sdk) as a `Processor`:

Hyp3 InSAR Processor takes a pair of `reference_granule_id` and a `secondary_granule_id` to generate an interferogram. To automate the pair selection process:


In [ ]:
from insarscript.utils import select_pairs, plot_pair_network
pair_stacks, B = select_pairs(filter_results, max_degree=5) # We set the maximum connections to 5 to limit interferograms
fig = plot_pair_network(pair_stacks, B)

If the network is healthy without any disconnections, you are ready to submit your pairs:

In [ ]:
from insarscript import Processor
for (path, frame), pairs in pair_stacks.items():
    processor = Processor.create('Hyp3_InSAR', pairs=pairs, workdir=f'/content')
    batch = processor.submit()
    processor.save()

This process will generate `hyp3_jobs.json` under your work directory, which contains the jobID submitted to the hyp3 server, the processing will take roughly 30 mins for 100 interferograms depends on the ASF server load

The job script will looks like:

In [ ]:
import json
with open('/content/hyp3_jobs.json', 'r') as f:
    data = json.load(f)
print(json.dumps(data, indent=4))

You may download the job file to check back the progress later:

In [ ]:
from google.colab import files
files.download('/content/hyp3_jobs.json')

To check the job processing status

  - If you still under same session:

In [ ]:
batchs = processor.refresh()

 - If you starts a new session and need to read from saved job file

In [ ]:
from google.colab import files
uploaded = files.upload()
for filename in uploaded:
    print(f"Saved file to /content/{filename}")
from insarscript import Processor
processor = Processor.create('Hyp3_InSAR', saved_job_path='/content/drive/MyDrive/hyp3_jobs.json')
batchs = processor.refresh()

Processing will take roughly 30 mins for every 100 interferograms depends on the ASF server load, once all jobs are completed, use `.download()` to download processed interferograms.

In [ ]:
processor.download()

Let's pretend the job has completed and download prepared files:

In [ ]:
!gdown --fuzzy "https://drive.google.com/file/d/1Tef1kLDKEPA11pNt8FhgLRM4aSnwOMTu/view?usp=sharing"
!gdown --fuzzy "https://drive.google.com/file/d/1NvCrDArsNU8zE82pG39vej3oaoXfZVFy/view?usp=sharing"

In [ ]:
!mkdir -p workdir/ERA5
!unzip clipped.zip -d /content/workdir
!unzip ERA5.zip -d /content/workdir

## SBAS (Analyzer)

After generated all unwrapped interferograms, time-series analysis is recommended for long term deformation monitoring. InSARScript currently support:

- **Mintpy**: an open-source Python package for InSAR time-series analysis.

InSARSciprt has wrapped Mintpy's SmallbaselineApp as an analyzer, to connect Mintpy with Hyp3 product and run time-series analysis:

(The SBAS process takes roughly 6 minutes to complete)

In [ ]:
from insarscript import Analyzer
workdir = '/content/workdir'
hyp3_sbas = Analyzer.create('Hyp3_SBAS', workdir=workdir, troposphericDelay_weatherDir='/content/workdir', reference_lalo='37.8412415,-112.8293839')
hyp3_sbas.prep_data()
hyp3_sbas.run()

To view the mean velocity map

In [ ]:
import rasterio
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline
from insarscript.utils import h5_to_raster

h5_to_raster('/content/workdir/velocity.h5')
raster_path = '/content/workdir/velocity_velocity.tif'

with rasterio.open(raster_path) as src:
    data = src.read(1)

nodata_value = -9999
data[data == nodata_value] = np.nan

vmin, vmax = np.nanpercentile(data, [0.1, 99.9])

# Create figure with proper layout
fig, ax = plt.subplots(figsize=(12, 10))

# Set bad values (NaN) to appear as light gray
cmap = plt.cm.turbo
cmap.set_bad(color='lightgray', alpha=0.3)

# Plot the data
im = ax.imshow(data, cmap=cmap, vmin=vmin, vmax=vmax, interpolation='bilinear')

# Add colorbar on the right
cbar = plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
cbar.set_label('Velocity (m/year)', fontsize=12, rotation=270, labelpad=20)
cbar.ax.tick_params(labelsize=10)

# Styling
ax.set_title('InSAR Velocity Map', fontsize=16, fontweight='bold', pad=15)
ax.axis('off')

plt.tight_layout()
plt.show()

To view the time series of specific location

In [ ]:
from ipywidgets import interact, IntSlider
import h5py
import matplotlib.pyplot as plt
import numpy as np
from datetime import datetime
import rasterio

# Read timeseries data
with h5py.File('/content/workdir/timeseries.h5', 'r') as f:
    timeseries = f['timeseries'][:]
    dates = f['date'][:]

date_list = [datetime.strptime(d.decode() if isinstance(d, bytes) else str(d), '%Y%m%d')
             for d in dates]
timeseries[timeseries == -9999] = np.nan

# Read velocity map
with rasterio.open('/content/workdir/velocity_velocity.tif') as src:
    velocity = src.read(1)

velocity[velocity == -9999] = np.nan

# Set your custom default values here
DEFAULT_X = 313  # Change this to your desired X pixel
DEFAULT_Y = 318  # Change this to your desired Y pixel

@interact(
    x=IntSlider(
        min=0,
        max=timeseries.shape[2]-1,
        value=DEFAULT_X,
        description='X pixel:'
    ),
    y=IntSlider(
        min=0,
        max=timeseries.shape[1]-1,
        value=DEFAULT_Y,
        description='Y pixel:'
    )
)
def plot_at_pixel(x, y):
    ts = timeseries[:, y, x]

    # Make timeseries start from 0 (relative to first date)
    ts_normalized = ts - ts[0]

    fig, axes = plt.subplots(1, 2, figsize=(16, 5))

    # Timeseries
    axes[0].plot(date_list, ts_normalized, 'o-', linewidth=2.5, markersize=7,
                 color='steelblue', markeredgecolor='navy')
    axes[0].axhline(y=0, color='red', linestyle='--', alpha=0.5, linewidth=2)
    axes[0].grid(True, alpha=0.3)
    axes[0].set_xlabel('Date', fontsize=12, fontweight='semibold')
    axes[0].set_ylabel('Displacement (m)', fontsize=12, fontweight='semibold')
    axes[0].set_title(f'Timeseries at pixel ({x}, {y})', fontsize=14, fontweight='bold')
    axes[0].tick_params(axis='x', rotation=45)

    # Add stats box
    mean_disp = np.nanmean(ts_normalized)
    std_disp = np.nanstd(ts_normalized)
    final_disp = ts_normalized[-1]
    axes[0].text(0.02, 0.98,
                f'Mean: {mean_disp:.2f} m\nStd: {std_disp:.2f} m\nFinal: {final_disp:.2f} m',
                transform=axes[0].transAxes, fontsize=10, verticalalignment='top',
                bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.7))

    # Velocity map
    vmin, vmax = np.nanpercentile(velocity, [0.1, 99.9])
    cmap = plt.cm.turbo
    cmap.set_bad(color='lightgray', alpha=0.3)

    im = axes[1].imshow(velocity, cmap=cmap, vmin=vmin, vmax=vmax, interpolation='bilinear')
    axes[1].plot(x, y, 'w*', markersize=22, markeredgewidth=2.5, markeredgecolor='black')
    axes[1].set_title('Mean Velocity Map', fontsize=14, fontweight='bold')
    axes[1].axis('off')

    cbar = plt.colorbar(im, ax=axes[1], fraction=0.046, pad=0.04)
    cbar.set_label('Velocity (m/year)', fontsize=11, rotation=270, labelpad=20)
    cbar.ax.tick_params(labelsize=10)

    plt.tight_layout()
    plt.show()